In [ ]:
#Necessary imports for fine tuning our embedding model 
!pip install -q sentence-transformers pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00


In [ ]:
# uploaded the cleaned dataset for further process
from google.colab import files
uploaded = files.upload()


Saving quotes.csv to quotes.csv/quotes.csv


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [7]:
df = pd.read_csv("quotes.csv/quotes.csv") #here i loaded the cleaned data from local pc

In [ ]:
##I created a helper function in the utils of main project but using it directly here otherwise project would become complex.
# Create InputExample pairs - (this was the function i made)
# This is main function here we are teaching our model that quote is sementically related to tags and authors
###So this function will help in our streamlit UI to place the quote higher when the user will search quote related tags or authors the quote will be given first priority
def create_input_examples(df):
    return [
        InputExample(texts=[row['quote'], f"{row['tags']} {row['author']}"], label=1.0)
        for _, row in df.iterrows()
    ]


In [ ]:
#Here we are applying the same function on the our dataframe (cleaned data or basically quotes.csv)
train_examples = create_input_examples(df)

In [9]:
# Dataloader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

In [11]:
model = SentenceTransformer('all-MiniLM-L6-v2')
train_loss = losses.CosineSimilarityLoss(model) # loss used here is cosine similarlty as it fits the problem statement
#using the all-MiniLM-L6-V2 model here

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)], 
    epochs=3,  # increased for better learning,
    show_progress_bar=True, # Shows the progress bar
    weight_decay=0.01,  # add regularization to avoid overfitting
    optimizer_params={'lr': 2e-5},  # smaller learning rate helps fine-tuning
    use_amp=True,  # mixed-precision training (faster, uses less GPU memory)
    #not using checkpoints here as the task is simple
)


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jackalnuri6 (jackalnuri6-jatinml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
#We saved the model in the google colab
model.save("fine-tuned-model-rag")


In [15]:
## Downloading the model locally here
!zip -r fine-tuned-model-rag.zip fine-tuned-model-rag
from google.colab import files
files.download("fine-tuned-model-rag.zip")

  adding: fine-tuned-model-rag/ (stored 0%)
  adding: fine-tuned-model-rag/tokenizer_config.json (deflated 73%)
  adding: fine-tuned-model-rag/modules.json (deflated 62%)
  adding: fine-tuned-model-rag/2_Normalize/ (stored 0%)
  adding: fine-tuned-model-rag/tokenizer.json (deflated 71%)
  adding: fine-tuned-model-rag/config_sentence_transformers.json (deflated 34%)
  adding: fine-tuned-model-rag/sentence_bert_config.json (deflated 4%)
  adding: fine-tuned-model-rag/1_Pooling/ (stored 0%)
  adding: fine-tuned-model-rag/1_Pooling/config.json (deflated 57%)
  adding: fine-tuned-model-rag/model.safetensors (deflated 8%)
  adding: fine-tuned-model-rag/README.md (deflated 65%)
  adding: fine-tuned-model-rag/config.json (deflated 48%)
  adding: fine-tuned-model-rag/vocab.txt (deflated 53%)
  adding: fine-tuned-model-rag/special_tokens_map.json (deflated 80%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>